In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from datetime import datetime, timedelta
import selenium.webdriver.common.keys
import pandas as pd
from bs4 import BeautifulSoup

In [20]:
# Install the ChromeDriver executable and start a Chrome browser using Selenium
driver = webdriver.Chrome()


In [21]:
# Navigate to the webpage
driver.get('https://merolagani.com/Floorsheet.aspx')


In [22]:
import os
import csv


file_name = "progress.csv"

# Check if the file exists
if not os.path.isfile(file_name):
    # If the file doesn't exist, create a new CSV file with headers
    with open(file_name, mode='w', newline='') as file:
        writer = csv.writer(file)
        # Write headers to the CSV file
    print(f"The file '{file_name}' has been created.")
else:
    print(f"The file '{file_name}' already exists.")

The file 'progress.csv' already exists.


In [29]:
def extract_page_data(date, page,total_pages,target_row=0 ):
    
    # Let's use BeautifulSoup to parse the page source
    soup = BeautifulSoup(driver.page_source, 'html.parser')

     # Let's use BeautifulSoup to parse the page source
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Extract the data you need from the current page
    # Adjust the selectors based on your HTML structure
    data = []
    rows = soup.select('.table-bordered tbody tr')

    # Find the index of the row with the target value
    if target_row != 0:
        target_index = next((index for index, row in enumerate(rows) if row.find_all('td')[0].get_text(strip=True) == str(target_row)), None)
        if target_index is not None:
            # Start enumeration from the next row
            rows = rows[target_index + 1:]

    # Iterate through rows starting from the specified target_row or the next row
    for index, row in enumerate(rows, start=target_row):
        columns = row.find_all('td')
        row_data = [column.get_text(strip=True) for column in columns] + [date]
        data.append(row_data)
        print(f"Adding data for row {index + 1}")

    return data


In [30]:
# Assuming the "Items per page" filter is a dropdown with the class 'ng-untouched'
# Wait for the dropdown to be present on the page
#wait = WebDriverWait(driver, 10)
#items_per_page_dropdown = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'ng-untouched')))
#items_per_page_dropdown.click()
# Find the option with value '500' and click on it
#option_500 = wait.until(EC.element_to_be_clickable((By.XPATH, '//option[@value="500"]')))
#option_500.click()
#filter_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'box__filter--search')))
#filter_button.click()
#print(driver.page_source)
#driver.implicitly_wait(2)


In [31]:
# Calculate the specific date (e.g., start from one week ago)
start_date_str = '01/21/2024'
start_date = datetime.strptime(start_date_str , '%m/%d/%Y')
date_array = []


# Get the current date
end_date = datetime.now()

# Iterate through the date range
while start_date <= end_date:
    # Format the current date as MM/DD/YYYY and print
    current_date_str = start_date.strftime('%m/%d/%Y')
#     date_input.clear()
#     date_input.send.keys(current_date_str)
    date_array.append(  current_date_str )
     
    # Move to the next date
    start_date += timedelta(days=1)
print(date_array)

['01/21/2024', '01/22/2024']


In [32]:
# Define a function to click the next page button
def click_next_page():
    try:
        # Find the next page element and click it
        next_page = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//a[text()="Next"]'))
        )
        next_page.click()
        return True
    except Exception as e:
        print(f"Error clicking next page: {e}")
        return False


In [33]:
csv_data = []
all_data = [] 
def scrape_data_for_date_range(date_array, page_number=0,target_row=0):
    num_pages_to_scrape = 2
    for date_str in date_array:
        try:
            print(f"Getting data from {date_str}")
            
            # Your existing code for setting the date, searching, and getting the data
            # ...

            # Loop through pages
            for current_page_num in range(page_number, page_number + num_pages_to_scrape):
                print(f"Scraping data from page {current_page_num + 1}")

                # Extract data from the current page
                current_data = extract_page_data(date_str, current_page_num + 1,num_pages_to_scrape,target_row)
                all_data.extend(current_data)

                # Click the next page
                if not click_next_page():
                    print("No more pages to scrape.")
                    break

        except Exception as e:
            print(f"Error scraping data for {date_str} and page {page_number + 1}: {e}")

def insertdata (date,data,page,total_entries):
        maximum_entries_from_db = 501
        print(f"hello+{data[0]}")
#         try execute the query 
# if success ok 
# check first or last 
#  append to csv at end with loading status
# if last replace csv with completed status 
# if duplicate unique key error 
# check the query again get the maximum value of entries and try to navigate to the certain page and insert the data after that
        if (total_entries!= maximum_entries_from_db):
            page = maximum_entries_from_db//500
            rows = maximum_entries_from_db%500
            scrape_data_for_date_range(date_array, page_number=page,target_row=data[0])
    
# if last page last entry than update csv to done 
# if first page entry than update csc to started
#             # Read the existing CSV file into a list of lists
#     with open(file, 'r', newline='') as file:
#         data = list(csv.reader(file))

#     # Check if the last row has the target value in the first column
#     if data and data[-1][0] == date:
#         # Replace the last row with the new row
#         data[-1] = new_row
#     else:
#         # If the target value is not found in the last row, append a new row
#         data.append(new_row)

#     # Write the modified data back to the CSV file
#     with open(file, 'w', newline='') as file:
#         csv.writer(file).writerows(data)
#         print(data)
#         writer.writerow(date,data[0],"pending")



In [34]:
scrape_data_for_date_range(date_array)

Getting data from 01/21/2024
Scraping data from page 1
Adding data for row 2
Adding data for row 3
Adding data for row 4
Adding data for row 5
Adding data for row 6
Adding data for row 7
Adding data for row 8
Adding data for row 9
Adding data for row 10
Adding data for row 11
Adding data for row 12
Adding data for row 13
Adding data for row 14
Adding data for row 15
Adding data for row 16
Adding data for row 17
Adding data for row 18
Adding data for row 19
Adding data for row 20
Adding data for row 21
Adding data for row 22
Adding data for row 23
Adding data for row 24
Adding data for row 25
Adding data for row 26
Adding data for row 27
Adding data for row 28
Adding data for row 29
Adding data for row 30
Adding data for row 31
Adding data for row 32
Adding data for row 33
Adding data for row 34
Adding data for row 35
Adding data for row 36
Adding data for row 37
Adding data for row 38
Adding data for row 39
Adding data for row 40
Adding data for row 41
Adding data for row 42
Adding dat

Scraping data from page 2
Adding data for row 2
Adding data for row 3
Adding data for row 4
Adding data for row 5
Adding data for row 6
Adding data for row 7
Adding data for row 8
Adding data for row 9
Adding data for row 10
Adding data for row 11
Adding data for row 12
Adding data for row 13
Adding data for row 14
Adding data for row 15
Adding data for row 16
Adding data for row 17
Adding data for row 18
Adding data for row 19
Adding data for row 20
Adding data for row 21
Adding data for row 22
Adding data for row 23
Adding data for row 24
Adding data for row 25
Adding data for row 26
Adding data for row 27
Adding data for row 28
Adding data for row 29
Adding data for row 30
Adding data for row 31
Adding data for row 32
Adding data for row 33
Adding data for row 34
Adding data for row 35
Adding data for row 36
Adding data for row 37
Adding data for row 38
Adding data for row 39
Adding data for row 40
Adding data for row 41
Adding data for row 42
Adding data for row 43
Adding data for 

Getting data from 01/22/2024
Scraping data from page 1
Adding data for row 2
Adding data for row 3
Adding data for row 4
Adding data for row 5
Adding data for row 6
Adding data for row 7
Adding data for row 8
Adding data for row 9
Adding data for row 10
Adding data for row 11
Adding data for row 12
Adding data for row 13
Adding data for row 14
Adding data for row 15
Adding data for row 16
Adding data for row 17
Adding data for row 18
Adding data for row 19
Adding data for row 20
Adding data for row 21
Adding data for row 22
Adding data for row 23
Adding data for row 24
Adding data for row 25
Adding data for row 26
Adding data for row 27
Adding data for row 28
Adding data for row 29
Adding data for row 30
Adding data for row 31
Adding data for row 32
Adding data for row 33
Adding data for row 34
Adding data for row 35
Adding data for row 36
Adding data for row 37
Adding data for row 38
Adding data for row 39
Adding data for row 40
Adding data for row 41
Adding data for row 42
Adding dat

Scraping data from page 2
Adding data for row 2
Adding data for row 3
Adding data for row 4
Adding data for row 5
Adding data for row 6
Adding data for row 7
Adding data for row 8
Adding data for row 9
Adding data for row 10
Adding data for row 11
Adding data for row 12
Adding data for row 13
Adding data for row 14
Adding data for row 15
Adding data for row 16
Adding data for row 17
Adding data for row 18
Adding data for row 19
Adding data for row 20
Adding data for row 21
Adding data for row 22
Adding data for row 23
Adding data for row 24
Adding data for row 25
Adding data for row 26
Adding data for row 27
Adding data for row 28
Adding data for row 29
Adding data for row 30
Adding data for row 31
Adding data for row 32
Adding data for row 33
Adding data for row 34
Adding data for row 35
Adding data for row 36
Adding data for row 37
Adding data for row 38
Adding data for row 39
Adding data for row 40
Adding data for row 41
Adding data for row 42
Adding data for row 43
Adding data for 

In [17]:

# Set the number of pages you want to scrape
span_element = soup.find('span', id='ctl00_ContentPlaceHolder1_PagerControl1_litRecords')


NameError: name 'soup' is not defined

In [18]:
# Close the browser
driver.quit()

In [ ]:
# Print the collected data
print("Collected Data:")
for row in all_data:
    print(row)